In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [3]:
# Document load
from populate_db import load_documents, split_documents

docs = load_documents()
print(f"Length of docs : {len(docs)}")
#print(docs[0])

# Split documents
chunks = split_documents(docs)
print(f"Length of chunks : {len(chunks)}")
#print(chunks[0])

Length of docs : 495
Length of chunks : 1543


In [4]:
#Update CHROMA DB if need to be updated
from populate_db import add_to_chroma_db, clear_chroma_db

#clear_chroma_db()
add_to_chroma_db(chunks)

Number of existing documents in DB: 1543
No new documents to add


In [5]:
# RAG Test not using Agent, just Query
from custom_rag_tool import query_rag
from IPython.display import Markdown

query_response = query_rag("5차 회의에서는 무슨 주제로 무제한 토론을 진행했나요?")

Markdown(query_response.content)

Human: 
Answer the question based only on the following context:

무제한 토론을  실 시함에  따라  오늘  예 정되었 던  대정부 질문은  실시 하기  어렵 게  되 었습니
다.  따라 서  국무총 리  및  국무 위원들 께서는  퇴장 하셔도  좋습 니다.
  무제한 토론에  앞서  몇  가지  안내말 씀  드 리겠습 니다.
  국회법  제106조의2제4항에  따라  무 제한토 론을  실시하 는  본 회의는  토론  종결  선포  전까
지  산 회하지  아 니하고  회의를  계 속하도 록  되어  있 습니다 .  따라 서  무제 한토론  중에는  자
정이  경과하 여도  차수 를  변경 하지  아니 하고  계속  본회의 를  진행 하게  됩니 다.  또한  국회
법  제 106조의2제 3 항에  따라  의원  한  분당  1회에  한 정하여  무제한 토론을  하실  수  있습니
다.
  같은  조  제6항  및  제7항 에  따라  더  이상  토론 할  의원 이  없거나  무제한 토론  종결  동의
가  가결되 면  무 제한토 론의  종결을  선포 하고  해당  안건은  지체  없이  표결 하게  됩니다 .
  그리고  같은  조  제8항 에  따라  무제 한토론 을  실 시하는  중에  회기 가  종료 되는  때에는  무

---

60  제415회－제4차(2024년7월2일)
◯본회의장 의석표

---

ਘੌ

제 415 회국회
(임시회)국회본회의회의록
(임시회의록)제 5 호
국  회  사  무  처
  2024년7월3일(수) 오후 3시
〔국회법  제106조의2의  규정에  의한  무제한토론  실시로  7월3일  개의하여  7월4 일  산회하였음〕
    의사일정
1.  순직  해병  수사  방해  및  사건  은폐  등의  진 상규명 을  위한  특별검 사의  임명  등에  관한  
법 률안(박찬대  의원  대표 발의) (의안 번 호  2200038)
2.  경제에  관한  질문
    상정된 

5차 회의에서는 "순직 해병 수사 방해 및 사건 은폐 등의 진상규명을 위한 특별검사의 임명 등에 관한 법률안"을 주제로 무제한 토론을 진행했습니다.

In [6]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from custom_rag_tool import CustomChromaDBRagTool

web_search_tool = SerperDevTool()
rag_tool = CustomChromaDBRagTool()

In [43]:
retriever_agent = Agent(
    role="Retriever",
    goal="Retrieve the relevaent information from the vectorstore to answer the question",
    backstory=(
        "You are an assistant for question-answering tasks."
        "Use the information present in the retrieved context to answer the question."
        "You have to provide a clear concise answer."
    ),
    verbose=True,
    allow_delegation=False,
)

retriever_task = Task(
    description=(
        "Use the rag_tool, extract information for the question {question}."
    ),
    expected_output=(
        "Set of relevant documents and passages that can be used to answer the question {question}."
        "Return a clear and consise text as response."
        "Please give all answers in Korean."
    ),
    agent=retriever_agent,
    tools=[rag_tool],
)

In [44]:
grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
)

grader_task = Task(
    description=(
        "Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=(
        "Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
        "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
        "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
        "Please provide explanation about the answer."
        "Please give all answers in Korean."
        #"Do not provide any preamble or explanations except for 'yes' or 'no'."
    ),
    agent=grader_agent,
    context=[retriever_task],
)

In [45]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
)

hallucination_task = Task(
    description=(
        "Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."
    ),
    expected_output=(
        "Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
        "Respond 'yes' if the answer is in useful and contains fact about the question asked."
        "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
        "Please provide explanation about the answer."
        "Please give all answers in Korean."
        #"Do not provide any preamble or explanations except for 'yes' or 'no'."
    ),
    agent=hallucination_grader,
    context=[grader_task],
)

In [46]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant then generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
)

answer_task = Task( 
    description=(
        "Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
        "If the answer is 'yes' return a clear and concise answer with Korean."
        "If the answer is 'no' then perform a 'websearch' and return the response"
    ),
    expected_output=(
        "Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
        "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
        "Otherwise respond as 'Sorry! unable to find a valid response'."
        "Please give all answers in Korean."
    ),  
    context=[hallucination_task],
    agent=answer_grader,
    tools=[web_search_tool],
)

In [47]:
rag_crew = Crew(
    agents=[retriever_agent, grader_agent, hallucination_grader, answer_grader],
    tasks=[retriever_task, grader_task, hallucination_task, answer_task],
    memory=True,
    verbose=True,
)

2024-07-19 15:24:28,125 - 8389020672 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [48]:
inputs = {"question":"대한민국 22대 415회 국회본회의 5차 회의에서는 무슨 주제로 무제한 토론을 진행했나요?"}

In [49]:
result = rag_crew.kickoff(inputs=inputs)

 [2024-07-19 15:24:30][DEBUG]: == Working Agent: Retriever
 [2024-07-19 15:24:30][INFO]: == Starting Task: Use the rag_tool, extract information for the question 대한민국 22대 415회 국회본회의 5차 회의에서는 무슨 주제로 무제한 토론을 진행했나요?.


> Entering new CrewAgentExecutor chain...
Thought: To answer the question about the topic of the unlimited debate at the 22nd National Assembly of South Korea, 415th plenary session, 5th meeting, I need to retrieve the relevant information from the Chroma DB.

Action: Custom Chroma DB Rag Tool
Action Input: {"query": "대한민국 22대 415회 국회본회의 5차 회의 무제한 토론 주제"} 

ਘੌ

제 415 회국회
(임시회)국회본회의회의록
(임시회의록)제 5 호
국  회  사  무  처
  2024년7월3일(수) 오후 3시
〔국회법  제106조의2의  규정에  의한  무제한토론  실시로  7월3일  개의하여  7월4 일  산회하였음〕
    의사일정
1.  순직  해병  수사  방해  및  사건  은폐  등의  진 상규명 을  위한  특별검 사의  임명  등에  관한  
법 률안(박찬대  의원  대표 발의) (의안 번 호  2200038)
2.  경제에  관한  질문
    상정된 안건
o  의 사 진 행 발 언 · ······· ········ ········ ········ ········ ········ ········ · ······· ········ ········ ········ ········ ········ ···